![image](../images/kdd24-logo-small.jpeg)

# Hands-on Tutorial
## Domain-Driven LLM Development: Insights into RAG and Fine-Tuning Practices
### Lab 1.2: Advanced RAG
#### Summary: 
This lab focused on enhancing the Retrieval-Augmented Generation (RAG) process by implementing and comparing four advanced optimizations against a baseline Naive RAG approach. The optimizations were:

- Sentence Chunking
- Hypothetical Question Generation
- Query Decomposition
- Hypothetical Question + Query Decomposition


#### Load the naive RAG performace as baseline

In [ ]:
!pip install pandas matplotlib

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_barchart(series_names, series_data, x_labels):
    """
    Plot bar charts for multiple series with x-axis labels and legends.

    :param series_names: List of names for each data series (used as legends).
    :param series_data: List of lists, where each inner list is a series of double numbers.
    :param x_labels: List of labels for the x-axis.
    """
    plt.figure(figsize=(10, 4))
    
    # Number of series to plot
    num_series = len(series_data)
    
    # Bar width depends on the number of series
    bar_width = 0.8 / num_series
    
    # X-axis positions for the bars
    x = range(len(x_labels))
    
    # Plot each series
    for i, (name, data) in enumerate(zip(series_names, series_data)):
        plt.bar([pos + i * bar_width for pos in x], data, bar_width, label=name)
    
    # Add labels, title, and legend
    plt.xlabel('Categories')
    plt.ylabel('Score')
    plt.title('Scores by Series')
    plt.xticks([pos + bar_width * (num_series / 2 - 0.5) for pos in x], x_labels)
    plt.legend()
    
    # Display the chart
    plt.tight_layout()
    plt.show()

file_path = '../lab-data/naive_rag_result.csv'
df = pd.read_csv(file_path)
metrics = df.columns[-3:]

# Prepare the data
series_names = [f"Naive RAG {name}" for name in metrics.tolist()]
series_data = [df[metric].tolist() for metric in metrics]

# Plot individual series bar charts
for i in range(3):
    plot_barchart([series_names[i]], [series_data[i]], list(range(len(series_data[i]))))

# Calculate the average for each metric
average_scores = [sum(data) / len(data) for data in series_data]

# Plot the average scores bar chart
plot_barchart(['Naive RAG Average Scores'], [average_scores], metrics.tolist())


In [ ]:
import fitz

In [ ]:
# Function to extract text using PyMuPDF
def extract_text_from_pdf_mupdf(pdf_path):
    text = ""
    document = fitz.open(pdf_path)
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
    return text

In [ ]:
# Extract the text from the PDF using PyMuPDF
pdf_text_mupdf = extract_text_from_pdf_mupdf("../lab-data/ENERGOUSCORP_03_16_2017-EX-10.24-STRATEGIC ALLIANCE AGREEMENT.PDF").replace('\xa0', ' ')
pdf_text_mupdf  # Displaying the first 2000 characters to get an overview of the content

#### Sentence Chunking + Sentence Window Retrieval Optimization

Purpose: Improve the retrieval accuracy by breaking down documents into smaller, more focused chunks (typically sentences).

Implementation: Documents are split into sentences, and each sentence is embedded separately. During the retrieval phase, queries are matched against these sentence-level embeddings, and then retrieve the surrounding sentences (similar idea as sliding window) to maintain the context cohesiveness.

Outcome: This technique generally improves the precision of retrieved content, ensuring that the most relevant parts of the documents are used in generation.

In [ ]:
def sentence_chunker(text):
    """
    Chunk text into sentences.
    """
    return [f"{sentence}." for sentence in text.split(".")]

sentences = sentence_chunker(pdf_text_mupdf)
sentences

In [ ]:
def sentence_window_builder(sentences, window_size):
    def sentence_length(sentence):
        """Helper function to calculate the length of a sentence."""
        return len(sentence)

    windows = []
    window_left_index = []
    window_right_index = []

    for i in range(len(sentences)):
        current_sentence = sentences[i]
        current_length = sentence_length(current_sentence)
        left_context = []
        right_context = []
        total_length = sentence_length(current_sentence)
        
        # Expand left context
        left_index = i - 1
        while left_index >= 0 and total_length + sentence_length(sentences[left_index]) - current_length // 2 <= window_size // 2:
            left_context.insert(0, sentences[left_index])
            total_length += sentence_length(sentences[left_index])
            left_index -= 1
        
        # Expand right context
        right_index = i + 1
        while right_index < len(sentences) and total_length + sentence_length(sentences[right_index]) - current_length // 2 <= window_size:
            right_context.append(sentences[right_index])
            total_length += sentence_length(sentences[right_index])
            right_index += 1
        
        # Build the window for the current sentence
        window = left_context + [current_sentence] + right_context
        windows.append(' '.join(window))
        window_left_index.append(left_index + 1)
        window_right_index.append(right_index - 1)
    
    return windows, window_left_index, window_right_index

window_size = 1024
windows, window_left_index, window_right_index = sentence_window_builder(sentences, window_size)
for i, w in enumerate(result):
    print(f"Window {i+1}: {w}")
    print(f"Window Range: [{window_left_index[i]}: {window_right_index[i]}]")

In [ ]:
from pyepsilla import vectordb
## connect to vectordb
db = vectordb.Client(
  host='localhost',
  port='8888'
)

In [ ]:
db.unload_db("kdd_lab1_advanced_rag")
db.load_db(db_name="kdd_lab1_advanced_rag", db_path="/tmp/kdd_lab1_advanced_rag")

In [ ]:
db.use_db(db_name="kdd_lab1_advanced_rag")
db.drop_table("SentenceWindow")
db.create_table(
  table_name="SentenceWindow",
  table_fields=[
    {"name": "ID", "dataType": "INT", "primaryKey": True},
    {"name": "Sentence", "dataType": "STRING"},
    {"name": "Left", "dataType": "INT"},
    {"name": "Right", "dataType": "INT"},
    {"name": "Embedding", "dataType": "VECTOR_FLOAT", "dimensions": 512}
  ]
)

In [ ]:
# Initialize the Bedrock client
client = boto3.client('bedrock-runtime', region_name='us-west-2')
# Function to embed text
def embed_text(input_text):
    # Create the request payload
    payload = {
        "inputText": input_text,
        "dimensions": 512,  # Specify the desired dimension size
        "normalize": True  # Whether to normalize the output embeddings
    }

    # Invoke the model
    response = client.invoke_model(
        body=json.dumps(payload),
        modelId='amazon.titan-embed-text-v2:0',  # Specify the Titan embedding model
        accept='application/json',
        contentType='application/json'
    )

    # Get the embedding result
    response_body = json.loads(response['body'].read())
    embedding = response_body.get('embedding')
    return embedding

# Print the embedding
print(embed_text(sentences[0]))

In [ ]:
records = [
    {
        "ID": index,
        "Sentence": sentences[index],
        "Left": window_left_index[index],
        "Right": window_right_index[index],
        "Embedding": embed_text(sentences[index])
    }
    for index in range(len(sentences))
]
records[:2]

In [ ]:
db.insert("SentenceWindow", records)

In [ ]:
def sentence_window_retriever(table_name, question, top_k):
    code, resp = db.query(
        table_name=table_name,
        query_field="Embedding",
        query_vector=embed_text(question),
        limit=top_k
    )
    id_list = []
    for item in resp["result"]:
        for id in range(item["Left"], item["Right"] + 1):
            if id not in id_list:  # Check if the ID is already in the list
                id_list.append(id)  # Append the ID if it's not a duplicate

    code, resp = db.get(
        table_name=table_name,
        response_fields=["Sentence"],
        primary_keys=id_list
    )
    return id_list, [item["Sentence"] for item in resp["result"]]
sentence_window_retriever("SentenceWindow", "What's the agreement date?", 5)

In [ ]:
import re
def sentence_window_rag(table_name, question):
    ids, sentences = sentence_window_retriever(table_name, question, 5)
    docs_str = "".join(sentences)
    prompt = f"""
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.

Your answer should be grounded by the information provided in the documents below.
Don't make up answers.
Don't explain your thought process.
Directly answer the question in concise way.

<documents>
{docs_str}
</documents>
<</SYS>>

{question}[/INST]
"""
    cleaned_response = re.sub(r'</?[^>]+>|\[/?(INST|SYS)\]', '', generate(prompt))
    return prompt, cleaned_response

data_augmented_prompt, answer = sentence_window_rag("SentenceWindow", "What's the agreement date?")

print(answer)

In [ ]:
import pandas as pd

# Load the CSV file
file_path = '../lab-data/ENERGOUSCORP_qa.csv'
df = pd.read_csv(file_path)

# Assuming the CSV has 'Question' and 'Answer' columns
questions = df['question'].tolist()
answers = df['answer'].tolist()

questions, answers

In [ ]:
data_augmented_prompts = []
generated_answers = []
for question in questions:
    data_augmented_prompt, generated_answer = sentence_window_rag("SentenceWindow", question)
    data_augmented_prompts.append(data_augmented_prompt)
    generated_answers.append(generated_answer)

generated_answers

In [ ]:
from continuous_eval.metrics.generation.text import DeterministicAnswerCorrectness
from sentence_transformers import SentenceTransformer, util

evaluation_results = []
for i in range(len(questions)):
    ground_truth_embedding = embed_text(answers[i])
    answer_embedding = embed_text(generated_answers[i])
    cosine_similarity = util.cos_sim(ground_truth_embedding, answer_embedding)[0][0].item()
    
    datum = {
        "answer": generated_answers[i],
        "ground_truth_answers": [
            answers[i]
        ]
    }
    metric = DeterministicAnswerCorrectness()
    eval_result = metric(**datum)
    evaluation_results.append({
        "question": data_augmented_prompts[i],
        "ref_answer": answers[i],
        "response": generated_answers[i],
        "semantic_similarity": cosine_similarity,
        "token_overlap_recall": eval_result["token_overlap_recall"],
        "rouge_l_recall": eval_result["rouge_l_recall"]
    })

evaluation_results

In [ ]:
import csv

# Specify the file name
csv_file = '../lab-data/sentence_window_rag_result.csv'

# Write the data to a CSV file
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=evaluation_results[0].keys())
    writer.writeheader()
    writer.writerows(evaluation_results)

csv_file

In [ ]:
file_path = '../lab-data/sentence_window_rag_result.csv'
df = pd.read_csv(file_path)
metrics = df.columns[-3:]

# Prepare the data
sentence_window_series_names = [f"Sentence Window RAG {name}" for name in metrics.tolist()]
sentence_window_series_data = [df[metric].tolist() for metric in metrics]

# Plot individual series bar charts
for i in range(3):
    plot_barchart(
        [series_names[i], sentence_window_series_names[i]],
        [series_data[i], sentence_window_series_data[i]],
        list(range(len(series_data[i])))
    )

# Calculate the average for each metric
sentence_window_average_scores = [sum(data) / len(data) for data in sentence_window_series_data]

# Plot the average scores bar chart
plot_barchart(['Naive RAG Average Scores', 'Sentence Window RAG Average Scores'], [average_scores, sentence_window_average_scores], metrics.tolist())

#### Hypothetical Questions Optimization

Purpose: Enhance retrieval relevance by transforming documents into hypothetical questions that are more likely to match user question semantically.

Implementation: Use LLM to generate a few hypothetical questions from each chunk. Embed the questions in vector database for retrieval, and when rendering the final prompt, use the corresponding original chunks instead of the hypothetical questions as knowledge.

Outcome: This approach increases the specificity and relevance of the retrieved content, improving the context provided to the language model.

In [ ]:
import boto3
import json
import re

def generate(prompt):
    # Create the request payload
    payload = {
        "prompt": prompt,
        "temperature": 0,  # Adjust the randomness of the output
        "max_gen_len": 128
    }

    # Initialize the Bedrock runtime client
    client = boto3.client('bedrock-runtime', region_name='us-west-2')

    # Invoke the model
    response = client.invoke_model(
        modelId='meta.llama3-8b-instruct-v1:0',
        contentType='application/json',
        accept='application/json',
        body=json.dumps(payload)
    )
    
    byte_response = response['body'].read()
    json_string = byte_response.decode('utf-8')

    # Get the chat response
    response_body = json.loads(json_string)
    chat_response = response_body.get('generation')

    return chat_response


def generate_hypothetical_questions(question):
    prompt = f"""
<s>[INST] <<SYS>>
Given the following text, generate a list of hypothetical questions that could be asked about the content, each in a single line. The questions should focus on key details, definitions, and information present in the text.
<</SYS>>
<text>
{question}
</text>
<questions>
[/INST]
"""
    cleaned_response = re.sub(r'</?[^>]+>', '', generate(prompt))
    return [question for question in cleaned_response.split('\n') if len(question) > 10]

questions = generate_hypothetical_questions("""Exhibit 10.24 \n \n[***]\nCertain confidential information contained in this document, marked by brackets, has been omitted and filed separately with the\nSecurities and Exchange Commission pursuant to Rule 24b-2 of the Securities Exchange Act of 1934, as amended.\n \nEXECUTION VERSION\n \nSTRATEGIC ALLIANCE AGREEMENT\n \nTHIS STRATEGIC ALLIANCE AGREEMENT (“Agreement”) is made and entered into as of November 6, 2016 (the “Effective Date”) by\nand between Dialog Semiconductor (UK) Ltd., a corporation organized under the laws of England and Wales, having its principal office at 100\nLongwater Avenue, Green Park, Reading, RG2 6GP, United Kingdom (“DIALOG”) and Energous Corporation, a Delaware corporation, having its\nprincipal office at 3590 North First Street, Suite 210, San Jose, CA 95134 (“ENERGOUS”).\n \nWHEREAS DIALOG is a supplier of mixed-signal semiconductor products;\n \nWHEREAS ENERGOUS is a supplier of uncoupled wirefree charging systems, including antennas, semiconductors, firmware, software,\nalgorithms, and sensors;""")

print(questions)

In [ ]:
def chunk_text(text, chunk_size, overlap):
    """
    Chunk text into smaller segments with a specified chunk size and overlap.

    Parameters:
    - text (str): The text to be chunked.
    - chunk_size (int): The size of each chunk.
    - overlap (int): The number of characters that overlap between chunks.

    Returns:
    - List[str]: A list of text chunks.
    """
    if chunk_size <= overlap:
        raise ValueError("Chunk size must be greater than overlap")

    chunks = []
    start = 0
    end = chunk_size

    while start < len(text):
        chunk = text[start:end]
        chunks.append(chunk)
        start += chunk_size - overlap
        end = start + chunk_size

    return chunks

chunks = chunk_text(pdf_text_mupdf, 1024, 256)
print(len(chunks))
chunks[:2]

In [ ]:
db.drop_table("HypotheticalQuestions")
db.create_table(
  table_name="HypotheticalQuestions",
  table_fields=[
    {"name": "Question", "dataType": "STRING"},
    {"name": "ChunkID", "dataType": "INT"},
    {"name": "Embedding", "dataType": "VECTOR_FLOAT", "dimensions": 512}
  ]
)

In [ ]:
from tqdm import tqdm

records = []
for index in tqdm(range(len(chunks)), desc="Processing chunks"):
    questions = generate_hypothetical_questions(chunks[index])
    for question in questions:
        records.append({
            "Question": question,
            "ChunkID": index,
            "Embedding": embed_text(question)
        })

print(records[:2])

In [ ]:
db.insert("HypotheticalQuestions", records)

In [ ]:
def hypothetical_question_retriever(table_name, question, top_k):
    code, resp = db.query(
        table_name=table_name,
        query_field="Embedding",
        query_vector=embed_text(question),
        limit=top_k
    )
    chunk_list = []
    for item in resp["result"]:
        if item["ChunkID"] not in chunk_list:  # Check if the ID is already in the list
            chunk_list.append(item["ChunkID"])  # Append the ID if it's not a duplicate

    return chunk_list, [chunks[id] for id in chunk_list]
hypothetical_question_retriever("HypotheticalQuestions", "What's the agreement date?", 15)

In [ ]:
def hypothetical_question_rag(table_name, question):
    ids, docs = hypothetical_question_retriever(table_name, question, 15)
    docs_str = "------------------------\n"
    for doc in docs:
        docs_str += doc + "\n------------------------\n"
    prompt = f"""
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.

Your answer should be grounded by the information provided in the documents below.
Don't make up answers.
Don't explain your thought process.
Directly answer the question in concise way.
Don't give answers to 

<documents>
{docs_str}
</documents>
<</SYS>>

{question}[/INST]
"""
    cleaned_response = re.sub(r'</?[^>]+>|\[/?(INST|SYS)\]', '', generate(prompt))
    return prompt, cleaned_response

data_augmented_prompt, answer = hypothetical_question_rag("HypotheticalQuestions", "What's the agreement date?")

print(answer)

In [ ]:
# Load the CSV file
file_path = '../lab-data/ENERGOUSCORP_qa.csv'
df = pd.read_csv(file_path)

# Assuming the CSV has 'Question' and 'Answer' columns
questions = df['question'].tolist()
answers = df['answer'].tolist()

questions, answers

In [ ]:
data_augmented_prompts = []
generated_answers = []
for index in tqdm(range(len(questions)), desc="Answering questions"):
    question = questions[index]
    data_augmented_prompt, generated_answer = hypothetical_question_rag("HypotheticalQuestions", question)
    data_augmented_prompts.append(data_augmented_prompt)
    generated_answers.append(generated_answer)

generated_answers

In [ ]:
evaluation_results = []
for i in range(len(questions)):
    ground_truth_embedding = embed_text(answers[i])
    answer_embedding = embed_text(generated_answers[i])
    cosine_similarity = util.cos_sim(ground_truth_embedding, answer_embedding)[0][0].item()
    
    datum = {
        "answer": generated_answers[i],
        "ground_truth_answers": [
            answers[i]
        ]
    }
    metric = DeterministicAnswerCorrectness()
    eval_result = metric(**datum)
    evaluation_results.append({
        "question": data_augmented_prompts[i],
        "ref_answer": answers[i],
        "response": generated_answers[i],
        "semantic_similarity": cosine_similarity,
        "token_overlap_recall": eval_result["token_overlap_recall"],
        "rouge_l_recall": eval_result["rouge_l_recall"]
    })

evaluation_results

In [ ]:
import csv

# Specify the file name
csv_file = '../lab-data/hypothetical_question_rag_result.csv'

# Write the data to a CSV file
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=evaluation_results[0].keys())
    writer.writeheader()
    writer.writerows(evaluation_results)

csv_file

In [ ]:
file_path = '../lab-data/hypothetical_question_rag_result.csv'
df = pd.read_csv(file_path)
metrics = df.columns[-3:]

# Prepare the data
hypothetical_question_series_names = [f"Hypothetical Question RAG {name}" for name in metrics.tolist()]
hypothetical_question_series_data = [df[metric].tolist() for metric in metrics]

# Plot individual series bar charts
for i in range(3):
    plot_barchart(
        [series_names[i], hypothetical_question_series_names[i]],
        [series_data[i], hypothetical_question_series_data[i]],
        list(range(len(series_data[i])))
    )

# Calculate the average for each metric
hypothetical_question_average_scores = [sum(data) / len(data) for data in hypothetical_question_series_data]

# Plot the average scores bar chart
plot_barchart(['Naive RAG Average Scores', 'Hypothetical Question RAG Average Scores'], [average_scores, hypothetical_question_average_scores], metrics.tolist())

#### Query Decomposition
Purpose: Handle complex queries by breaking them down into simpler sub-queries, each targeting a specific aspect of the original question.

Implementation: A complex query is decomposed into multiple, more manageable sub-queries. Each sub-query is processed independently through the retrieval system, and the results are combined before being passed to the language model.

Outcome: This technique helps in better addressing multifaceted queries by ensuring that all relevant aspects are covered in the final response.

In [ ]:
def decompose_question(question):
    prompt = f"""
<s>[INST] <<SYS>>
Given the following complex query, analyze whether it requires decomposition into simpler subqueries. If the query is already straightforward and can be answered directly, do not decompose it. If the query is complex or involves multiple steps, break it down into a sequence of subqueries that can be addressed individually. Each subquestion in a line.
<</SYS>>
<question>
{question}
</question>
[/INST]
<sub_questions>
"""
    cleaned_response = re.sub(r'</?[^>]+>|\[/?(INST|SYS)\]', '', generate(prompt))
    return [question for question in cleaned_response.split('\n') if len(question) > 10][:4]

sub_questions = decompose_question("""Does one party have the right to terminate or is consent or notice required of the counterparty if such party undergoes a change of control, such as a merger, stock sale, transfer of all or substantially all of its assets or business, or assignment by operation of law?""")

print(sub_questions)

In [ ]:
db.drop_table("NaiveRAG")
db.create_table(
  table_name="NaiveRAG",
  table_fields=[
    {"name": "ID", "dataType": "INT", "primaryKey": True},
    {"name": "Doc", "dataType": "STRING"},
    {"name": "Embedding", "dataType": "VECTOR_FLOAT", "dimensions": 512}
  ]
)

In [ ]:
records = [
    {
        "ID": index,
        "Doc": text,
        "Embedding": embed_text(text)
    }
    for index, text in enumerate(chunks)
]
records[:2]

In [ ]:
db.insert("NaiveRAG", records)

In [ ]:
def basic_retriever(table_name, question, top_k):
    code, resp = db.query(
        table_name=table_name,
        query_field="Embedding",
        query_vector=embed_text(question),
        limit=top_k
    )
    return resp["result"]
basic_retriever("NaiveRAG", "What's the agreement date?", 5)

In [ ]:
def naive_rag(table_name, question):
    docs = basic_retriever(table_name, question, 5)
    docs_str = "------------------------\n"
    for doc in docs:
        docs_str += doc["Doc"] + "\n------------------------\n"
    prompt = f"""
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.

Your answer should be grounded by the information provided in the documents below.
Don't make up answers.
Don't explain your thought process.
Directly answer the question in concise way.

<documents>
{docs_str}
</documents>
<</SYS>>
[/INST]
<question>
{question}
</question>
"""
    cleaned_response = re.sub(r'</?[^>]+>|\[/?(INST|SYS)\]', '', generate(prompt))
    return prompt, cleaned_response

data_augmented_prompt, answer = naive_rag("NaiveRAG", "What's the agreement date?")

answer

In [ ]:
def sub_question_resolver(sub_questions):
    question_answer = ""
    for sub_question in sub_questions:
        data_augmented_prompt, answer = naive_rag("NaiveRAG", sub_question)
        question_answer += f"Sub-Question: {sub_question}\nAnswer: {answer}\n------------------------\n"
    return question_answer
sub_question_resolver(decompose_question("Does one party have the right to terminate or is consent or notice required of the counterparty if such party undergoes a change of control, such as a merger, stock sale, transfer of all or substantially all of its assets or business, or assignment by operation of law?"))

In [ ]:
def sub_query_rag(question):
    sub_questions_qa = sub_question_resolver(decompose_question(question))
    prompt = f"""
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.

Your answer should be grounded by the information provided in the groud_truth section below.
Don't make up answers.
Don't explain your thought process.
Directly answer the question in concise way.

<groud_truth>
{sub_questions_qa}
</groud_truth>
<</SYS>>
[/INST]
<question>
{question}
</question>
<answer>
"""
    cleaned_response = re.sub(r'</?[^>]+>|\[/?(INST|SYS)\]', '', generate(prompt))
    return prompt, cleaned_response

data_augmented_prompt, answer = sub_query_rag("Does one party have the right to terminate or is consent or notice required of the counterparty if such party undergoes a change of control, such as a merger, stock sale, transfer of all or substantially all of its assets or business, or assignment by operation of law?")

data_augmented_prompt, answer

In [ ]:
# Load the CSV file
file_path = '../lab-data/ENERGOUSCORP_qa.csv'
df = pd.read_csv(file_path)

# Assuming the CSV has 'Question' and 'Answer' columns
questions = df['question'].tolist()
answers = df['answer'].tolist()

questions, answers

In [ ]:
data_augmented_prompts = []
generated_answers = []
for index in tqdm(range(len(questions)), desc="Answering questions"):
    question = questions[index]
    data_augmented_prompt, generated_answer = sub_query_rag(question)
    data_augmented_prompts.append(data_augmented_prompt)
    generated_answers.append(generated_answer)

generated_answers

In [ ]:
evaluation_results = []
for i in range(len(questions)):
    ground_truth_embedding = embed_text(answers[i])
    answer_embedding = embed_text(generated_answers[i])
    cosine_similarity = util.cos_sim(ground_truth_embedding, answer_embedding)[0][0].item()
    
    datum = {
        "answer": generated_answers[i],
        "ground_truth_answers": [
            answers[i]
        ]
    }
    metric = DeterministicAnswerCorrectness()
    eval_result = metric(**datum)
    evaluation_results.append({
        "question": data_augmented_prompts[i],
        "ref_answer": answers[i],
        "response": generated_answers[i],
        "semantic_similarity": cosine_similarity,
        "token_overlap_recall": eval_result["token_overlap_recall"],
        "rouge_l_recall": eval_result["rouge_l_recall"]
    })

evaluation_results

In [ ]:
import csv

# Specify the file name
csv_file = '../lab-data/query_decomposition_rag_result.csv'

# Write the data to a CSV file
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=evaluation_results[0].keys())
    writer.writeheader()
    writer.writerows(evaluation_results)

csv_file

In [ ]:
file_path = '../lab-data/query_decomposition_rag_result.csv'
df = pd.read_csv(file_path)
metrics = df.columns[-3:]

# Prepare the data
query_decomposition_series_names = [f"Query Decomposition RAG {name}" for name in metrics.tolist()]
query_decomposition_series_data = [df[metric].tolist() for metric in metrics]

# Plot individual series bar charts
for i in range(3):
    plot_barchart(
        [series_names[i], query_decomposition_series_names[i]],
        [series_data[i], query_decomposition_series_data[i]],
        list(range(len(series_data[i])))
    )

# Calculate the average for each metric
query_decomposition_average_scores = [sum(data) / len(data) for data in query_decomposition_series_data]

# Plot the average scores bar chart
plot_barchart(['Naive RAG Average Scores', 'Query Decomposition RAG Average Scores'], [average_scores, query_decomposition_average_scores], metrics.tolist())

#### Query Decomposition + Hypothetical Questions
Purpose: Combine the strengths of both Hypothetical Question Generation and Query Decomposition for handling complex queries with targeted specificity.

Implementation: The query is first decomposed into sub-queries, and each sub-query is then using the Hypothetical Question pipeline to get the answer. This hybrid approach aims to maximize the relevance and coverage of the retrieved content.

Outcome: This method provides the most comprehensive and contextually enriched responses, particularly for complex queries, by ensuring both broad coverage and targeted retrieval.

In [ ]:
def sub_question_resolver_hypothetical_question(sub_questions):
    question_answer = ""
    for sub_question in sub_questions:
        data_augmented_prompt, answer = hypothetical_question_rag("HypotheticalQuestions", sub_question)
        question_answer += f"Sub-Question: {sub_question}\nAnswer: {answer}\n------------------------\n"
    return question_answer
sub_question_resolver_hypothetical_question(decompose_question("Does one party have the right to terminate or is consent or notice required of the counterparty if such party undergoes a change of control, such as a merger, stock sale, transfer of all or substantially all of its assets or business, or assignment by operation of law?"))

In [ ]:
def sub_query_rag_with_hypothetical_question(question):
    ids, docs = hypothetical_question_retriever("HypotheticalQuestions", question, 15)
    docs_str = "------------------------\n"
    for doc in docs:
        docs_str += doc + "\n------------------------\n"
    sub_questions_qa = sub_question_resolver_hypothetical_question(decompose_question(question))
    prompt = f"""
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.

Your answer should be grounded by the information provided in the documents and groud_truth section below.
Don't make up answers.
Don't explain your thought process.
Directly answer the question in concise way.

<documents>
{docs_str}
</documents>

<groud_truth>
{sub_questions_qa}
</groud_truth>
<</SYS>>
[/INST]
<question>
{question}
</question>
<answer>
"""
    cleaned_response = re.sub(r'</?[^>]+>|\[/?(INST|SYS)\]', '', generate(prompt))
    return prompt, cleaned_response

data_augmented_prompt, answer = sub_query_rag_with_hypothetical_question("What's the agreement date?")

data_augmented_prompt, answer

In [ ]:
# Load the CSV file
file_path = '../lab-data/ENERGOUSCORP_qa.csv'
df = pd.read_csv(file_path)

# Assuming the CSV has 'Question' and 'Answer' columns
questions = df['question'].tolist()
answers = df['answer'].tolist()

questions, answers

In [ ]:
data_augmented_prompts = []
generated_answers = []
for index in tqdm(range(len(questions)), desc="Answering questions"):
    question = questions[index]
    data_augmented_prompt, generated_answer = sub_query_rag_with_hypothetical_question(question)
    data_augmented_prompts.append(data_augmented_prompt)
    generated_answers.append(generated_answer)

generated_answers

In [ ]:
evaluation_results = []
for i in range(len(questions)):
    ground_truth_embedding = embed_text(answers[i])
    answer_embedding = embed_text(generated_answers[i])
    cosine_similarity = util.cos_sim(ground_truth_embedding, answer_embedding)[0][0].item()
    
    datum = {
        "answer": generated_answers[i],
        "ground_truth_answers": [
            answers[i]
        ]
    }
    metric = DeterministicAnswerCorrectness()
    eval_result = metric(**datum)
    evaluation_results.append({
        "question": data_augmented_prompts[i],
        "ref_answer": answers[i],
        "response": generated_answers[i],
        "semantic_similarity": cosine_similarity,
        "token_overlap_recall": eval_result["token_overlap_recall"],
        "rouge_l_recall": eval_result["rouge_l_recall"]
    })

evaluation_results

In [ ]:
# Specify the file name
csv_file = '../lab-data/advanced_rag_result.csv'

# Write the data to a CSV file
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=evaluation_results[0].keys())
    writer.writeheader()
    writer.writerows(evaluation_results)

csv_file

In [ ]:
file_path = '../lab-data/advanced_rag_result.csv'
df = pd.read_csv(file_path)
metrics = df.columns[-3:]

# Prepare the data
qd_and_hq_series_names = [f"Query Decomposition + Hypothetical Question RAG {name}" for name in metrics.tolist()]
qd_and_hq_series_data = [df[metric].tolist() for metric in metrics]

# Plot individual series bar charts
for i in range(3):
    plot_barchart(
        [series_names[i], qd_and_hq_series_names[i]],
        [series_data[i], qd_and_hq_series_data[i]],
        list(range(len(series_data[i])))
    )

# Calculate the average for each metric
qd_and_hq_average_scores = [sum(data) / len(data) for data in qd_and_hq_series_data]

# Plot the average scores bar chart
plot_barchart(['Naive RAG Average Scores', 'Query Decomposition + Hypothetical Question RAG Average Scores'], [average_scores, qd_and_hq_average_scores], metrics.tolist())